# Financial Screen Scraping

Average investors constantly get sapped by get-rich-quick schemes or the well-meaning advice of friends. Generally lacking in financial literacy, they're prone to fall victim to a good story without doing rigorous analysis on securities. 

The problems with rigorous analysis:
1. It's hard! 
2. It takes too much time! 
3. It's expensive, the data is hard to access!

All of these things are true, but it doesn't have to be that way. 

This notebook is a guide through using the Discounted Cashflow (DCF) valuation model to value your basket of securities. This model uses past cash flows to determine whether a stock is overvalued or undervalued and by how much. 

Accessing the data to feed the model can be highly time consuming, so this model also crawls Yahoo Finance for the relevant datapoints, cleasn and processes them, and provides them in a table format that makes them easy to compare. 

Yahoo Finance contains a bevy of other information that may be used to create other types of models. Once you understand how to collect and clean the data, you can process it with various models and compare their performance!

In [9]:
import pandas as pd #data processing
from tqdm import tqdm #progress bars
import pygsheets #export to Google Sheets
import numpy as np #numerical processing
pd.set_option('max_columns', 500)

In [2]:
def npv_fcf(fcf, discount_rate, years, growth_rate, multiplier, growth_decline_rate, cash_on_hand, total_debt):
    '''Calculates net present value of a free cash flow over a given time period'''
    
    npv = [] #initialize a list
    
    #Add values depending on year, growth rate, growth decline, and discount rate
    for year in range(1, years+1):
        fcf = fcf * (1 + growth_rate)/((1 + discount_rate))
        growth_rate = growth_rate * (1-growth_decline_rate)
        
        final_fcf = fcf
    
        npv.append(fcf)
    
    max_year_fcf_value = final_fcf * multiplier #Year 10NPV Value
    npv.append(max_year_fcf_value) 
    
    npv.append(cash_on_hand)
    npv.append(-total_debt)
    
        
    return sum(npv)  

In [3]:
def trackr(stocks, margin_of_safety=.25, discount_rate=.1, growth_decline=.05, years=10, multiplier=12, yahoo_adjust=1000):
    '''trackr reads a dictionary of stocks and their associated company names and categories and returns today's Yahoo Finance 
    stats on the corresponding stock as a row of values that can be appended to a dataframe'''
    
    import datetime
    import pandas as pd
    from tqdm import tqdm
    import numpy as np
    
    #Error Handling
    if type(stocks) != dict:
        print('Error. You must enter these in a dictionary format, where the symbol is the key, name is first list value, and the subsequent list values are categories')
        return
    
    tickers = list(stocks.keys()) #convert to list in case single string entered
    quotes = [] #capture each entry as a row
    
    for ticker in tqdm(tickers):
        #Grab appropriate data based on tickers. This stores the appropriate URLs and then pulls data from the pages of Yahoo Finance
        
        quote = 'https://finance.yahoo.com/quote/{}?p-{}'.format(ticker, ticker) #Quote string
        cashflow = 'https://finance.yahoo.com/quote/{}/cash-flow?p={}'.format(ticker, ticker) #Cashflow string
        stat = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'.format(ticker, ticker) #Key Statistics String
        balance = 'https://finance.yahoo.com/quote/{}/balance-sheet?p={}&.tsrc=fin-srch'.format(ticker, ticker) #Balance Sheet String
        analysis = 'https://finance.yahoo.com/quote/{}/analysis?p={}'.format(ticker, ticker) #Analysis String
        quote_data = pd.read_html(quote) #grab the data that YAHOO has on the given stock
        cashflow_data = pd.read_html(cashflow) #grab the cashflow statement
        stat_data = pd.read_html(stat) #grab key stats
        balance_data = pd.read_html(balance) #grab balance sheet
        analysis_data = pd.read_html(analysis) #grab analysis data

        #The HTML tables parse into a list. It will take some cleanup to prepare the list for a dataframe
        #stock 1 gives us Previous Close, Open, Bid, Ask, Day's Range, 52 Week Range, Volume, and Avg. Volume
        #stock 2 gives us Market Cap, Beta (3Y Monthly), PE Ratio (TTM), EPS (TTM), Earnings Date, Forward Dividend & Yield, Ex-Dividend Date, 1y Target Est

        stock1 = quote_data[0].transpose() #Transpose data into meaningful arrangement
        stock1.columns = stock1.iloc[0] #set new header row
        stock1.drop(0, inplace=True) #drop the old

        stock2 = quote_data[1].transpose() #Transpose data into meaningful arrangement
        stock2.columns = stock2.iloc[0] #set new header row
        stock2.drop(0, inplace=True) #drop the old

        #combine them
        stock_cat = pd.concat([stock1, stock2], axis=1)
        
        #grab cashflow data for discounted cashflow calculation
        stock_cat['Cash Flow'] = np.where(cashflow_data[0].iloc[9][1] == '-', 
                                          cashflow_data[0].iloc[9][2],
                                          cashflow_data[0].iloc[9][1])
        stock_cat['CapEx'] = np.where(cashflow_data[0].iloc[11][1] == '-', 
                                          cashflow_data[0].iloc[11][2],
                                          cashflow_data[0].iloc[11][1])
        

        #add them up for Free Cash Flow calculation
        stock_cat['Free Cash Flow'] = int(stock_cat['Cash Flow']) + int(stock_cat['CapEx'])
        
        # Adds cash and cash equivalents with short term investments. Discards blank values that yahoo returns and goes back a year if needed
        
        stock_cat['Cash On Hand'] = np.where(balance_data[0].iloc[2,1] == '-',
                                             int(balance_data[0].iloc[2,2]) + int(balance_data[0].iloc[3,2].replace('-', '0')),
                                             int(balance_data[0].iloc[2,1].replace('-', '0')) + int(balance_data[0].iloc[3,1].replace('-', '0')))
        
        #grabs long term debt, filling in blank values where needed
        stock_cat['Long Term Debt'] = np.where(balance_data[0].iloc[21, 1] == '-',
                                               balance_data[0].iloc[21, 2],
                                               balance_data[0].iloc[21, 1])
        
        #Pulls Yahoo's version of shares outstanding, applying a different multiplier for billions vs millions
        stock_cat['Approx Shares Outstanding'] = np.where(stat_data[8].loc[2,1][-1] == 'B',
                                                          float(stat_data[8].loc[2,1][:-1]) * 1000000000,
                                                          float(stat_data[8].loc[2,1][:-1]) * 1000000)
        
        #pulls Yahoo's compiling of analyst 5 year estimate for growth
        stock_cat['Conservative Analyst 5y'] = (float(analysis_data[5].loc[4, ticker.upper()].split('%')[0])/100)*(1-margin_of_safety)
        
        
        # turn columns to numeric
        num_cols = ['Cash Flow', 'Free Cash Flow', 'CapEx', 'Cash On Hand', 'Long Term Debt', 'Approx Shares Outstanding', 'Conservative Analyst 5y',
                    'Previous Close', 'Open', 'Volume', 'Avg. Volume', 'Beta (3Y Monthly)', 'PE Ratio (TTM)', 'EPS (TTM)', '1y Target Est']
        for col in num_cols:
            stock_cat[col] = np.nan_to_num(pd.to_numeric(stock_cat[col], errors='coerce'))
            
        # convert to appropriate values by multiplying needed cols by 1000
        dollar_cols = ['Cash Flow', 'CapEx', 'Free Cash Flow', 'Cash On Hand', 'Long Term Debt']
        for col in dollar_cols:
            stock_cat[col] = stock_cat[col] * yahoo_adjust
        
        #Calculate net present value of the equity            
        stock_cat['DCF Value'] = npv_fcf(stock_cat['Free Cash Flow'], 
                                         discount_rate, 
                                         years, 
                                         stock_cat['Conservative Analyst 5y'],
                                         multiplier,
                                         growth_decline,
                                         stock_cat['Cash On Hand'],
                                         stock_cat['Long Term Debt']
                                         )
        
        #Compares the Discounted Cash Flow to share price so decisions can be made about what's overvalued and undervalued. Higher than 1 means undervalued
        stock_cat['DCF Share Price'] = stock_cat['DCF Value'] / stock_cat['Approx Shares Outstanding']
        stock_cat['DCF/Price Multiple'] = stock_cat['DCF Share Price']/stock_cat['Previous Close']
        
        #Give the period of reports using cash on hand report as proxy for data not reported
        stock_cat['Reporting Period'] = np.where(balance_data[0].iloc[2,1] == '-',
                                                 balance_data[0].iloc[0,2],
                                                 balance_data[0].iloc[0,1])
        stock_cat['Reporting Period'] = pd.to_datetime(stock_cat['Reporting Period'])
        
        #Preps needed categories for display
        stock_cat['Date'] = pd.to_datetime(datetime.date.today())
        stock_cat['Ticker'] = ticker.upper()
        stock_cat['Name'] = stocks[ticker][0]
        stock_cat['Categories'] = [stocks[ticker][1:]]

        #reorder the columns for more meaningful view
        stock_cat = stock_cat[list(stock_cat.columns[-4:]) + list(stock_cat.columns)[:-4]]
        
        #finally, add the entry to the list
        quotes.append(stock_cat.iloc[0])
    
    stock_frame = pd.DataFrame(quotes) #Turn these values into a dataframe
    stock_frame.replace('N/A (N/A)', np.nan, inplace=True) #properly indicate null values"
    stock_frame.set_index('Date', drop=True, inplace=True) #and prepare datetime indexing for better analysis
    
    return stock_frame #return a dataframe reflecting each entry

In [4]:
#Dictionary of stocks requested to track. Key is ticker symbol, list position 1 is name, and subsequent positions are categories to which the company belongs.

bk_stocks = {'aapl': ['Apple', 'Cell Phone'], 
             '005930.KS': ['Samsung', 'Cell Phone'],
             't': ['ATT', 'Cell Service Provider', 'ISP'],
             'vz': ['Verizon', 'Cell Service Provider'],
             'cmcsa': ['Comcast', 'ISP'],
             'chtr': ['Charter', 'ISP'],
             'duk': ['Duke Energy', 'Power Provider'],
             'ngg': ['National Grid', 'Power Provider'],
             'so': ['Southern Company', 'Power Provider'],
             'googl': ['Google', 'Search'],
             'fb': ['Facebook', 'Advertising'],
             'twtr': ['Twitter', 'Twitter'],
             'gsk': ['Glaxosmithkline', 'Toothpaste'],
             'cl': ['Colgate-Palmolive', 'Toothpaste'],
             'ul': ['Unilever', 'Soap'],
             'ip': ['International Paper', 'Paper'],
             'RDS-A': ['Royal Dutch Shell', 'Gas'],
             'xom': ['Exxon', 'Gas'],
             'cvx': ['Chevron', 'Gas'],
             'pg': ['Proctor & Gamble', 'Detergent'],
             'chd': ['Church and Dwight', 'Detergent'],
             'kdp': ['Kuerig Dr. Pepper', 'Coffee'],
             'sjm': ['JM Smucker Company', 'Coffee'],
             'sbux': ['Starbucks', 'Coffee'],
             'amzn': ['Amazon', 'Retail', 'Cloud Storage'],
             'wmt': ['Wal Mart', 'Retail', 'Grocery'],
             'tgt': ['Target', 'Retail'],
             'kr': ['Kroger', 'Kroger'],
             'msft': ['Microsoft', 'Cloud Storage'],
             'ibm': ['IBM', 'Cloud Storage'],
             'crm': ['Salesforce', 'Cloud Storage']}

In [5]:
dcf_trackr = trackr(bk_stocks)

100%|██████████| 31/31 [01:32<00:00,  2.78s/it]


In [6]:
quotes = dcf_trackr[['Ticker', 'Name', 'Categories', 'Previous Close', 'Market Cap', 'Cash Flow', 
                     'CapEx', 'Free Cash Flow', 'Cash On Hand', 'Long Term Debt', 'Approx Shares Outstanding', 
                     'Conservative Analyst 5y', 'DCF Value', 'DCF Share Price', 'DCF/Price Multiple', 'Reporting Period']]

In [8]:
#Publish to Google Sheets
sheet_auth = ''
sheet_name = 'y_finance_stocks'
gc = pygsheets.authorize(service_file=sheet_auth)
sh = gc.open(sheet_name)
wks = sh[0]
wks.set_dataframe(quotes.reset_index(), (1,1)) #specifies cell coordinates of upper leftmost cell